In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [33]:
student_data = pd.read_csv('../1.Data/student_data_filt.csv', index_col=False)
student_data_enrolled = pd.read_csv('../1.Data/student_data_holdout.csv', index_col=False)
data_mapping = pd.read_csv('../1.Data/data_type_mapping.csv', index_col=False)

#### Review Data Types, Association, and Scaled Impact

In [34]:
data_mapping.sort_values(by=['Impact'])

,Column,Data Type,Association Type,Association Result,Impact
16,Scholarship holder,Binary,Phi Coefficient,0.312380,High
1,Application mode,Categorical,Cramer's V,0.311989,High
14,Tuition fees up to date,Binary,Phi Coefficient,0.441309,High
3,Course,Categorical,Cramer's V,0.340179,High
29,Curricular units 2nd sem (grade),Continuous,Point Biserial,-0.605350,High
28,Curricular units 2nd sem (approved),Numerical-discrete,Point Biserial,-0.653995,High
15,Gender,Categorical,Cramer's V,0.251955,High
22,Curricular units 1st sem (approved),Numerical-discrete,Point Biserial,-0.554881,High
23,Curricular units 1st sem (grade),Continuous,Point Biserial,-0.519927,High
19,Curricular units 1st sem (credited),Numerical-discrete,Point Biserial,-0.046900,Low


#### Split Data into Train Test

In [35]:
#Data needs to be split into training and testing sets

#Split the data into features and target label
target = student_data['Target']
features = student_data.drop('Target', axis=1)

#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)


#### Encode Categorical Variables

In [36]:
#create a list of categorical columns
categorical_columns = data_mapping[data_mapping['Data Type'] == 'Categorical']['Column'].values

In [37]:
#Split the data into categorical and numerical features
X_train_cat = X_train[categorical_columns]
X_test_cat = X_test[categorical_columns]
X_train_num = X_train.drop(categorical_columns, axis=1)
X_test_num = X_test.drop(categorical_columns, axis=1)

In [38]:
#One hot encode the categorical data
#One hot encoding is used to convert categorical data, which is represented as strings, into numerical data
#This is done because most machine learning algorithms cannot handle categorical data
#One hot encoding creates a new column for each unique value in a categorical column

#keep track of the index of the training and testing sets
index_train = X_train_cat.index
index_test = X_test_cat.index

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train_cat)
X_test_encoded = encoder.transform(X_test_cat)

In [39]:
#Print the shape of the encoded data
print(X_train_encoded.shape)
print(X_test_encoded.shape)

(2722, 65)
(908, 65)


In [40]:
#Convert the encoded data into a dataframe
X_train_encoded = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out())
X_test_encoded = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out())
X_train_encoded.index = index_train
X_test_encoded.index = index_test

In [41]:
#Concatenate the encoded data with the numerical data
X_train = pd.concat([X_train_encoded, X_train_num], axis=1)
X_test = pd.concat([X_test_encoded, X_test_num], axis=1)

In [42]:
X_train.head()

,Marital status_Divorced,Marital status_Married,Marital status_Other_Marital,Marital status_Single,Application mode_DegreeExperienceApp,Application mode_FirstPhaseApp,Application mode_OtherApp,Application mode_Over23App,Application mode_SecondThirdPhaseApp,Application mode_TransferApp,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
3438,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,5,12,0,0.000000,0,12.7,3.7,-1.70
1322,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0,6,7,0,0.000000,0,8.9,1.4,3.51
3535,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0,1,6,12,6,13.625000,0,12.4,0.5,1.79
1047,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,6,7,6,12.857143,0,11.1,0.6,2.02
765,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2,0,5,12,0,0.000000,4,10.8,1.4,1.74


In [43]:
X_train.to_csv('../1.Data/TrainTest/X_train.csv', index=False)
X_test.to_csv('../1.Data/TrainTest/X_test.csv', index=False)
y_train.to_csv('../1.Data/TrainTest/y_train.csv', index=False)
y_test.to_csv('../1.Data/TrainTest/y_test.csv', index=False)


##### Removal of 2nd Semester Figures to determine early intervention opportunities

In [44]:
#identiy the columns that have '2nd' in them
second_semester = [col for col in student_data if '2nd' in col] 

In [45]:
second_semester

['Curricular units 2nd sem (credited)',
 'Curricular units 2nd sem (enrolled)',
 'Curricular units 2nd sem (evaluations)',
 'Curricular units 2nd sem (approved)',
 'Curricular units 2nd sem (grade)',
 'Curricular units 2nd sem (without evaluations)']

In [46]:
student_data_nosecond = student_data.drop(second_semester, axis=1)

In [47]:
#Data needs to be split into training and testing sets

#Split the data into features and target label
target = student_data_nosecond['Target']
features = student_data_nosecond.drop('Target', axis=1)

#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=36)

In [48]:
#create a list of categorical columns
categorical_columns = data_mapping[data_mapping['Data Type'] == 'Categorical']['Column'].values

In [49]:
#Split the data into categorical and numerical features
X_train_cat = X_train[categorical_columns]
X_test_cat = X_test[categorical_columns]
X_train_num = X_train.drop(categorical_columns, axis=1)
X_test_num = X_test.drop(categorical_columns, axis=1)
#One hot encode the categorical data
#One hot encoding is used to convert categorical data, which is represented as strings, into numerical data
#This is done because most machine learning algorithms cannot handle categorical data
#One hot encoding creates a new column for each unique value in a categorical column

#keep track of the index of the training and testing sets
index_train = X_train_cat.index
index_test = X_test_cat.index

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train_cat)
X_test_encoded = encoder.transform(X_test_cat)

In [50]:
#Print the shape of the encoded data
print(X_train_encoded.shape)
print(X_test_encoded.shape)

(2722, 65)
(908, 65)


In [51]:
#Convert the encoded data into a dataframe
X_train_encoded = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out())
X_test_encoded = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out())
X_train_encoded.index = index_train
X_test_encoded.index = index_test

In [52]:
#Concatenate the encoded data with the numerical data
X_train = pd.concat([X_train_encoded, X_train_num], axis=1)
X_test = pd.concat([X_test_encoded, X_test_num], axis=1)

In [53]:
X_train.to_csv('../1.Data/TrainTest/X_train_no2.csv', index=False)
X_test.to_csv('../1.Data/TrainTest/X_test_no2.csv', index=False)
y_train.to_csv('../1.Data/TrainTest/y_train_no2.csv', index=False)
y_test.to_csv('../1.Data/TrainTest/y_test_no2.csv', index=False)

#### Prepare Unseen Data to Match Shape of Training Data

In [54]:
student_data_enrolled['Marital status'].replace({1: 'Single', 2: 'Married', 4: 'Divorced', 3: 'Other_Marital', 5: 'Other_Marital', 6: 'Other_Marital'}, inplace=True)
student_data_enrolled['Gender'].replace({0: 'Female', 1: 'Male'}, inplace=True)
student_data_enrolled['Daytime/evening attendance']. replace({0: 'evening', 1: 'daytime'}, inplace=True)
student_data_enrolled['Course'].replace({
    1: 'Technology and Information Systems',
    2: 'Animation and Digital Art',
    3: 'Social Service',
    4: 'Agronomy',
    5: 'Communication Sciences',
    6: 'Vetinary',
    7: 'Technology and Information Systems',
    8: 'Equine Studies',
    9: 'Management',
    10: 'Social Service',
    11: 'Tourism', 
    12: 'Nursing',
    13: 'Oral Hygiene',
    14: 'Advertising and Marketing',
    15: 'Journalism and Communication',
    16: 'General Studies',
    17: 'Management'
}, inplace=True)
student_data_enrolled['Nacionality'].replace({
    1: 'Portuguese',
    2: 'Non-Portuguese',
    3: 'Non-Portuguese',
    4: 'Non-Portuguese',
    5: 'Non-Portuguese',
    6: 'Non-Portuguese',
    7: 'Non-Portuguese',
    8: 'Non-Portuguese',
    9: 'Non-Portuguese',
    10: 'Non-Portuguese',
    11: 'Non-Portuguese',
    12: 'Non-Portuguese',
    13: 'Non-Portuguese',
    14: 'Non-Portuguese',
    15: 'Non-Portuguese',
    16: 'Non-Portuguese',
    17: 'Non-Portuguese',
    18: 'Non-Portuguese',
    19: 'Non-Portuguese',
    20: 'Non-Portuguese',
    21: 'Non-Portuguese'
}, inplace=True)
student_data_enrolled['Previous qualification'].replace({
    1: 'S-SecondaryEducation',
    2: 'S-HigherEducation',
    3: 'S-HigherEducation',
    4: 'S-HigherEducation',
    5: 'S-HigherEducation',
    6: 'S-HigherEducation',
    7: 'S-SecondaryIncomplete',
    8: 'S-SecondaryIncomplete',
    9: 'S-SecondaryIncomplete',
    10: 'S-SecondaryIncomplete',
    11: 'S-SecondaryIncomplete',
    12: 'S-BasicEducation',
    13: 'S-BasicEducation',
    14: 'S-Technical',
    15: 'S-HigherEducation',
    16: 'S-Technical',
    17: 'S-HigherEducation'
}, inplace=True)
student_data_enrolled['Mother\'s qualification'].replace({
    1: 'M-SecondaryEducation',
    2: 'M-HigherEducation',
    3: 'M-HigherEducation',
    4: 'M-HigherEducation',
    5: 'M-HigherEducation',
    6: 'M-HigherEducation',
    7: 'M-SecondaryIncomplete',
    8: 'M-SecondaryIncomplete',
    9: 'M-SecondaryIncomplete',
    10: 'M-SecondaryIncomplete',
    11: 'M-SecondaryIncomplete',
    12: 'M-SecondaryIncomplete',
    13: 'M-SecondaryIncomplete',
    14: 'M-BasicEducation',
    15: 'M-SecondaryIncomplete',
    16: 'M-TechnicalProfessional',
    17: 'M-SecondaryIncomplete',
    18: 'M-SecondaryIncomplete',
    19: 'M-SecondaryIncomplete',
    20: 'M-SecondaryIncomplete',
    21: 'M-SecondaryIncomplete',
    22: 'M-TechnicalProfessional',
    23: 'M-TechnicalProfessional',
    24: 'M-SecondaryIncomplete',
    25: 'M-SecondaryIncomplete',
    26: 'M-SecondaryIncomplete',
    27: 'M-BasicEducation',
    28: 'M-BasicEducation',
    29: 'M-TechnicalProfessional',
    30: 'M-HigherEducation',
    31: 'M-TechnicalProfessional',
    32: 'M-TechnicalProfessional',
    33: 'M-HigherEducation',
    34: 'M-HigherEducation'
}, inplace=True)
student_data_enrolled['Father\'s qualification'].replace({
    1: 'F-SecondaryEducation',
    2: 'F-HigherEducation',
    3: 'F-HigherEducation',
    4: 'F-HigherEducation',
    5: 'F-HigherEducation',
    6: 'F-HigherEducation',
    7: 'F-SecondaryIncomplete',
    8: 'F-SecondaryIncomplete',
    9: 'F-SecondaryIncomplete',
    10: 'F-SecondaryIncomplete',
    11: 'F-SecondaryIncomplete',
    12: 'F-SecondaryIncomplete',
    13: 'F-SecondaryIncomplete',
    14: 'F-BasicEducation',
    15: 'F-SecondaryIncomplete',
    16: 'F-TechnicalProfessional',
    17: 'F-SecondaryIncomplete',
    18: 'F-SecondaryIncomplete',
    19: 'F-SecondaryIncomplete',
    20: 'F-SecondaryIncomplete',
    21: 'F-SecondaryIncomplete',
    22: 'F-TechnicalProfessional',
    23: 'F-TechnicalProfessional',
    24: 'F-SecondaryIncomplete',
    25: 'F-SecondaryIncomplete',
    26: 'F-SecondaryIncomplete',
    27: 'F-BasicEducation',
    28: 'F-BasicEducation',
    29: 'F-TechnicalProfessional',
    30: 'F-HigherEducation',
    31: 'F-TechnicalProfessional',
    32: 'F-TechnicalProfessional',
    33: 'F-HigherEducation',
    34: 'F-HigherEducation'
}, inplace=True)
student_data_enrolled['Mother\'s occupation'].replace({
    1: 'M-Student',
    2: 'M-AdministrativeManagement',
    3: 'M-IntellectualScientific',
    4: 'M-TechnicalSkilled',
    5: 'M-SupportAdministrative',
    6: 'M-ServiceSales',
    7: 'M-AgricultureFarming',
    8: 'M-TechnicalSkilled',
    9: 'M-TechnicalSkilled',
    10: 'M-UnskilledLabor',
    11: 'M-ArmedForces',
    12: 'M-OtherMiscellaneous',
    13: 'M-ArmedForces',
    14: 'M-ArmedForces',
    15: 'M-ArmedForces',
    16: 'M-ArmedForces',
    17: 'M-AdministrativeManagement',
    18: 'M-AdministrativeManagement',
    19: 'M-IntellectualScientific',
    20: 'M-IntellectualScientific',
    21: 'M-IntellectualScientific',
    22: 'M-IntellectualScientific',
    23: 'M-TechnicalSkilled',
    24: 'M-TechnicalSkilled',
    25: 'M-TechnicalSkilled',
    26: 'M-TechnicalSkilled',
    27: 'M-SupportAdministrative',
    28: 'M-SupportAdministrative',
    29: 'M-SupportAdministrative',
    30: 'M-ServiceSales',
    31: 'M-ServiceSales',
    32: 'M-ServiceSales',
    33: 'M-ServiceSales',
    34: 'M-AgricultureFarming',
    35: 'M-AgricultureFarming',
    36: 'M-TechnicalSkilled',
    37: 'M-TechnicalSkilled',
    38: 'M-TechnicalSkilled',
    39: 'M-TechnicalSkilled',
    40: 'M-TechnicalSkilled',
    41: 'M-TechnicalSkilled',
    42: 'M-TechnicalSkilled',
    43: 'M-OtherMiscellaneous',
    44: 'M-OtherMiscellaneous',
    45: 'M-OtherMiscellaneous',
    46: 'M-OtherMiscellaneous'
}, inplace=True)
student_data_enrolled['Father\'s occupation'].replace({
    1: 'F-Student',
    2: 'F-AdministrativeManagement',
    3: 'F-IntellectualScientific',
    4: 'F-TechnicalSkilled',
    5: 'F-SupportAdministrative',
    6: 'F-ServiceSales',
    7: 'F-AgricultureFarming',
    8: 'F-TechnicalSkilled',
    9: 'F-TechnicalSkilled',
    10: 'F-UnskilledLabor',
    11: 'F-ArmedForces',
    12: 'F-OtherMiscellaneous',
    13: 'F-ArmedForces',
    14: 'F-ArmedForces',
    15: 'F-ArmedForces',
    16: 'F-ArmedForces',
    17: 'F-AdministrativeManagement',
    18: 'F-AdministrativeManagement',
    19: 'F-IntellectualScientific',
    20: 'F-IntellectualScientific',
    21: 'F-IntellectualScientific',
    22: 'F-IntellectualScientific',
    23: 'F-TechnicalSkilled',
    24: 'F-TechnicalSkilled',
    25: 'F-TechnicalSkilled',
    26: 'F-TechnicalSkilled',
    27: 'F-SupportAdministrative',
    28: 'F-SupportAdministrative',
    29: 'F-SupportAdministrative',
    30: 'F-ServiceSales',
    31: 'F-ServiceSales',
    32: 'F-ServiceSales',
    33: 'F-ServiceSales',
    34: 'F-AgricultureFarming',
    35: 'F-AgricultureFarming',
    36: 'F-TechnicalSkilled',
    37: 'F-TechnicalSkilled',
    38: 'F-TechnicalSkilled',
    39: 'F-TechnicalSkilled',
    40: 'F-TechnicalSkilled',
    41: 'F-TechnicalSkilled',
    42: 'F-TechnicalSkilled',
    43: 'F-OtherMiscellaneous',
    44: 'F-OtherMiscellaneous',
    45: 'F-OtherMiscellaneous',
    46: 'F-OtherMiscellaneous'
}, inplace=True)
student_data_enrolled['Application mode'].replace({
    1: 'FirstPhaseApp',
    2: 'OtherApp',
    3: 'FirstPhaseApp',
    4: 'DegreeExperienceApp',
    5: 'OtherApp',
    6: 'OtherApp',
    7: 'FirstPhaseApp',
    8: 'SecondThirdPhaseApp',
    9: 'SecondThirdPhaseApp',
    10: 'OtherApp',
    11: 'OtherApp',
    12: 'Over23App',
    13: 'TransferApp',
    14: 'TransferApp',
    15: 'DegreeExperienceApp',
    16: 'TransferApp',
    17: 'DegreeExperienceApp',
    18: 'OtherApp'
}, inplace=True)

In [55]:
student_data_enrolled.head()

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,Single,SecondThirdPhaseApp,1,Social Service,daytime,S-SecondaryEducation,Portuguese,M-SecondaryIncomplete,F-BasicEducation,M-ServiceSales,...,0,6,14,2,11.000,0,10.8,1.4,1.74,Enrolled
1,Single,FirstPhaseApp,1,General Studies,daytime,S-SecondaryEducation,Portuguese,M-SecondaryIncomplete,F-BasicEducation,M-TechnicalSkilled,...,0,7,8,6,13.500,0,16.2,0.3,-0.92,Enrolled
2,Single,SecondThirdPhaseApp,4,Oral Hygiene,daytime,S-SecondaryEducation,Portuguese,M-SecondaryEducation,F-BasicEducation,M-SupportAdministrative,...,0,8,9,8,11.425,0,12.7,3.7,-1.70,Enrolled
3,Single,FirstPhaseApp,1,Social Service,daytime,S-SecondaryEducation,Portuguese,M-SecondaryIncomplete,F-BasicEducation,M-UnskilledLabor,...,0,6,12,4,11.000,0,7.6,2.6,0.32,Enrolled
4,Single,FirstPhaseApp,1,Vetinary,daytime,S-SecondaryEducation,Portuguese,M-SecondaryIncomplete,F-BasicEducation,M-TechnicalSkilled,...,0,5,7,4,13.000,0,9.4,-0.8,-3.12,Enrolled


In [56]:
features = student_data_enrolled.drop('Target', axis=1)
features_cat = features[categorical_columns]
features_num = features.drop(categorical_columns, axis=1)
index_features = features_cat.index


encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
features_encoded = encoder.fit_transform(features_cat)


#Convert the encoded data into a dataframe
features_encoded = pd.DataFrame(features_encoded, columns=encoder.get_feature_names_out())
features_encoded.index = index_features

#Concatenate the encoded data with the numerical data
enrolled_students = pd.concat([features_encoded, features_num], axis=1)


In [57]:
enrolled_students.head()

,Marital status_Divorced,Marital status_Married,Marital status_Other_Marital,Marital status_Single,Application mode_DegreeExperienceApp,Application mode_FirstPhaseApp,Application mode_OtherApp,Application mode_Over23App,Application mode_SecondThirdPhaseApp,Application mode_TransferApp,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,6,14,2,11.000,0,10.8,1.4,1.74
1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,7,8,6,13.500,0,16.2,0.3,-0.92
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0,0,8,9,8,11.425,0,12.7,3.7,-1.70
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,6,12,4,11.000,0,7.6,2.6,0.32
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2,0,5,7,4,13.000,0,9.4,-0.8,-3.12


In [59]:
different_columns = [col for col in X_train if col not in enrolled_students]

In [60]:
different_columns

["Mother's occupation_M-OtherMiscellaneous"]

In [61]:
enrolled_students["Mother's occupation_M-OtherMiscellaneous"] = 0

In [62]:
enrolled_students.to_csv('../1.Data/TrainTest/enrolled_students.csv', index=False)